In [1]:
from kafka import KafkaProducer
import threading
import json
import random
import time

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = 'bb3a6a894c14:9093'
TOPIC_PREFIX = 'tweet-'

In [2]:
def worker(user_id: int, topic: str, num_messages: int):

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            time.sleep(2)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrumpido por el usuario.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer cerrado.")

In [3]:
if __name__ == "__main__":
    NUM_MESSAGES = random.randint(100, 500)
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic, NUM_MESSAGES),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    print("Todos los hilos han terminado.")


[Thread-2] Iniciando, topic=tweet-2, mensajes=146
[Thread-1] Iniciando, topic=tweet-1, mensajes=146
[Thread-4] Iniciando, topic=tweet-4, mensajes=146
[Thread-3] Iniciando, topic=tweet-3, mensajes=146
[Thread-2] Sent to tweet-2: 46bbca5d-1b32-46e8-896e-295c8b117c7b
[Thread-1] Sent to tweet-1: 9f14198f-6219-42cb-9b3b-6b3b5f965c85
[Thread-4] Sent to tweet-4: 24be3ea0-7928-44df-9a4f-e48465882986
[Thread-3] Sent to tweet-3: 9215ada6-b8ce-4522-825f-c777cf00aab5
[Thread-2] Sent to tweet-2: b724f9c5-d387-47e0-ac70-b6b09f73fd5c
[Thread-1] Sent to tweet-1: 20dc1565-33a2-4382-99e6-5abcfb74c755
[Thread-4] Sent to tweet-4: 1db15504-3b8a-49b5-96ee-6b0bcf270ed0
[Thread-3] Sent to tweet-3: 419fe24a-8630-4b40-8642-7b40e6853526
[Thread-2] Sent to tweet-2: 4e151805-703e-4fdc-bc49-7ae3080f5335
[Thread-1] Sent to tweet-1: 9624af07-b01a-496c-856d-ca2f101ef686
[Thread-4] Sent to tweet-4: f58d6d64-99e4-4884-b216-95d86c2becc0
[Thread-3] Sent to tweet-3: 4bd6c826-5c6a-497a-aed5-e77cbefc3801
[Thread-2] Sent to t